In [ ]:
%pip install billboard.py


In [ ]:
import pickle
import billboard

import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import datetime
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
charts = []
charts.append(billboard.ChartData('hot-100'))
while len(charts) < 100:
  charts.insert(0, billboard.ChartData('hot-100', charts[0].previousDate))
  print('\r', len(charts), end="")

 100

In [ ]:
print(vars(charts[0][0]))

{'title': 'In My Feelings', 'artist': 'Drake', 'image': None, 'peakPos': 1, 'lastPos': 1, 'weeks': 11, 'rank': 1, 'isNew': False}


In [ ]:
x_train = []
y_train = []

artists = {}

for i in range(len(charts) - 1):
  x_train.append([])
  y_train.append([])
  for song in charts[i]:
    if song.artist not in artists:
      artists[song.artist] = random.random() * 10
    x_train[-1].extend([artists[song.artist], song.peakPos, song.lastPos, song.weeks, song.rank, song.isNew])
    found = False
    for c in charts[i+1]:
      if c.title == song.title:
        y_train[-1].append(c.rank)
        found = True
        break
    if not found:
      y_train[-1].append(101)


  #   if song in charts[i+1]:
  #     print('sc')
  # for song in charts[i+1]:
  #   if song.artist not in artists:
  #     artists[song.artist] = random.random() * 10
  #   y_train[-1].append(song.rank)

x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape, y_train.shape)

(99, 600) (99, 100)


In [ ]:
y_train[0:4]

In [ ]:
class Network(nn.Module):
    def __init__(self, in_size=64):
      super(Network, self).__init__()
      # 300 -> 300 -> batch norm -> dropout -> 150 -> 150 -> batch norm -> dropout -> 100 -> 100
      self.l1 = nn.Linear(in_size, 300)
      self.l2 = nn.Linear(300, 300)
      self.l3 = nn.BatchNorm1d(300, track_running_stats=False)
      self.l4 = nn.Dropout(0.5)
      self.l5 = nn.Linear(300, 150)
      self.l6 = nn.Linear(150, 150)
      self.l7 = nn.BatchNorm1d(150, track_running_stats=False)
      self.l8 = nn.Dropout(0.5)
      self.l9 = nn.Linear(150, 100)
      self.l10 = nn.Linear(100, 100) 

    def forward(self, x):
      x = F.relu(self.l1(x))
      x = F.relu(self.l2(x))
      x = self.l3(x)
      x = self.l4(x)
      x = F.relu(self.l5(x))
      x = F.relu(self.l6(x))
      x = self.l7(x)
      x = self.l8(x)
      x = F.relu(self.l9(x))
      x = F.relu(self.l10(x))

      return x

In [ ]:
net = Network(600)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
import math
import time
batchsize = 4
bestLoss = 1e15
for epoch in range(400):  # loop over the dataset multiple times
  startTime = time.time()
  num = len(x_train)
  running_loss = 0.0
  for i in range(0, len(x_train), batchsize):

    e = min(i + batchsize, len(x_train))
    inputs = torch.FloatTensor(x_train[i:e])
    y = torch.FloatTensor(y_train[i:e])

    labels = y

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    losses = []
    for o in range(len(outputs)):
      losses.append(criterion(outputs[o], y[o]))
    loss = losses[0]
    for l in losses[1:]: loss = loss + l
    # loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    dif = time.time()-startTime
    eta = round(dif/(i+1) * num - dif)

    # print statistics
    running_loss += loss.item()
    # print('\r', 'epoch', epoch, str(i) + '/' + str(num) + ', eta:', eta, ', loss:', round(running_loss / batchsize, 3), end="")
  # print()
  print('\r', 'epoch', epoch, str(i) + '/' + str(num) + ', Time:', round(time.time() - startTime), ', loss:', running_loss / batchsize, end="")
  if running_loss < bestLoss:
    torch.save(net.state_dict(), 'best.pt')
    print('\nsaved', running_loss/batchsize)
    bestLoss = running_loss


print('\nFinished Training')

 epoch 0 96/99, Time: 0 , loss: 100419.51025390625
saved 100419.51025390625
 epoch 1 96/99, Time: 0 , loss: 97937.05200195312
saved 97937.05200195312
 epoch 2 96/99, Time: 0 , loss: 91597.05834960938
saved 91597.05834960938
 epoch 3 96/99, Time: 0 , loss: 70521.64428710938
saved 70521.64428710938
 epoch 4 96/99, Time: 0 , loss: 33818.58544921875
saved 33818.58544921875
 epoch 5 96/99, Time: 0 , loss: 17524.03778076172
saved 17524.03778076172
 epoch 6 96/99, Time: 0 , loss: 10237.187850952148
saved 10237.187850952148
 epoch 7 96/99, Time: 0 , loss: 8047.673858642578
saved 8047.673858642578
 epoch 8 96/99, Time: 0 , loss: 7287.564071655273
saved 7287.564071655273
 epoch 10 96/99, Time: 0 , loss: 7058.025360107422
saved 7058.025360107422
 epoch 12 96/99, Time: 0 , loss: 6913.825881958008
saved 6913.825881958008
 epoch 16 96/99, Time: 0 , loss: 6760.942489624023
saved 6760.942489624023
 epoch 20 96/99, Time: 0 , loss: 6533.536468505859
saved 6533.536468505859
 epoch 40 96/99, Time: 0 , los

In [ ]:
print(net(torch.FloatTensor(x_train[0:4])))

In [ ]:
# torch.save(net, 'trainedNet.pth')
with open('artists.pickle', 'wb') as handle:
    pickle.dump(artists, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model = Network(600)
model = model.load_state_dict('bestNet.pt')
model.eval()
ex = torch.FloatTensor(np.array([x_train[-1], x_train[-1]]))
results = model(ex)[0]
print(results)
print(np.argsort(list(results)))

In [ ]:
# outputs = model(get_next())[0]
newPos = np.argsort(list(results))
ranking = []
for p in newPos:
  ranking.append(charts[-2][p])

for i in range(10):
  print(ranking[i], '||||', charts[-1][i])

'Cardigan' by Taylor Swift |||| 'Watermelon Sugar' by Harry Styles
'Whats Poppin' by Jack Harlow Featuring DaBaby, Tory Lanez & Lil Wayne |||| 'Rockstar' by DaBaby Featuring Roddy Ricch
'Rockstar' by DaBaby Featuring Roddy Ricch |||| 'Whats Poppin' by Jack Harlow Featuring DaBaby, Tory Lanez & Lil Wayne
'Blinding Lights' by The Weeknd |||| 'Blinding Lights' by The Weeknd
'Exile' by Taylor Swift Featuring Bon Iver |||| 'Roses' by SAINt JHN
'The 1' by Taylor Swift |||| 'My Future' by Billie Eilish
'Roses' by SAINt JHN |||| 'Savage Love (Laxed - Siren Beat)' by Jawsh 685 x Jason Derulo
'Watermelon Sugar' by Harry Styles |||| 'Cardigan' by Taylor Swift
'Savage' by Megan Thee Stallion Featuring Beyonce |||| 'Go Crazy' by Chris Brown & Young Thug
'Savage Love (Laxed - Siren Beat)' by Jawsh 685 x Jason Derulo |||| 'Blueberry Faygo' by Lil Mosey


In [ ]:
import billboard
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

charts = []
charts.append(billboard.ChartData('hot-100'))
charts.append(billboard.ChartData('hot-100', charts[0].previousDate))

previous = charts[0][:10]
current = charts[1][:10]

model = Network(600)
model.load_state_dict(torch.load('best.pt'))
model.eval()

artistFile = open('artists.pickle', 'rb')
artists = pickle.load(artistFile)

def get_next():
	x = []

	for song in charts[1]:
		x.extend([artists[song.artist], song.peakPos, song.lastPos, song.weeks, song.rank, song.isNew])

	return torch.FloatTensor(np.array([x, x]))

def predict():
	outputs = model(get_next())[0]
	newPos = np.argsort(list(outputs))
	ranking = []
	for p in newPos:
		ranking.append(charts[1][p])
	
	return ranking

In [ ]:
import billboard
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

class Network(nn.Module):
    def __init__(self, in_size=64):
      super(Network, self).__init__()
      # 300 -> 300 -> batch norm -> dropout -> 150 -> 150 -> batch norm -> dropout -> 100 -> 100
      self.l1 = nn.Linear(in_size, 300)
      self.l2 = nn.Linear(300, 300)
      self.l3 = nn.BatchNorm1d(300, track_running_stats=False)
      self.l4 = nn.Dropout(0.5)
      self.l5 = nn.Linear(300, 150)
      self.l6 = nn.Linear(150, 150)
      self.l7 = nn.BatchNorm1d(150, track_running_stats=False)
      self.l8 = nn.Dropout(0.5)
      self.l9 = nn.Linear(150, 100)
      self.l10 = nn.Linear(100, 100) 

    def forward(self, x):
      x = F.relu(self.l1(x))
      x = F.relu(self.l2(x))
      x = self.l3(x)
      x = self.l4(x)
      x = F.relu(self.l5(x))
      x = F.relu(self.l6(x))
      x = self.l7(x)
      x = self.l8(x)
      x = F.relu(self.l9(x))
      x = F.relu(self.l10(x))

      return x

charts = []
charts.append(billboard.ChartData('hot-100'))
charts.append(billboard.ChartData('hot-100', charts[0].previousDate))

previous = charts[0][:10]
current = charts[1][:10]

model = Network(600)
model.load_state_dict(torch.load('best.pt'))
model.eval()

artistFile = open('artists.pickle', 'rb')
artists = pickle.load(artistFile)

def get_next():
	x = []

	for song in charts[1]:
		x.extend([artists[song.artist], song.peakPos, song.lastPos, song.weeks, song.rank, song.isNew])

	return torch.FloatTensor(np.array([x, x]))

def predict():
	outputs = model(get_next())[0]
	newPos = np.argsort(list(outputs))
	ranking = []
	for p in newPos:
		ranking.append(charts[1][p])
	
	return ranking